# Demonstrating Visualization of OPERA DIST-ANN Product Layers
---
This notebook provides an overview of the basic functionality and utility of the OPERA **DIST-ANN** product, an near-global annual pixel-wise summary of vegetation change. Here, several of the available **DIST-ANN** rasters are visualized for a wildfire-affected area in northern California.

**<font color='red'>Note: This notebook uses provisional products, which may differ slightly from operational products. Please refer to [DIST product specification](https://d2pn8kiwq2w21t.cloudfront.net/documents/ProductSpec_DIST_HLS.pdf) for more information. </font>**

In [ ]:
### Library Imports
import hvplot.xarray
import geoviews as gv
import holoviews as hv

from bokeh.models import FixedTicker
hv.extension('bokeh')

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../../')
from src.dist_utils import *

### DIST Product Suite Background
---
The land Disturbance product suite (**DIST**) maps vegetation disturbance from Harmonized Landsat-8 and Sentinel-2 A/B (HLS) scenes. Disturbance is detected when vegetation cover decreases or spectral variation is outside a historical norm within an HLS pixel. Two DIST products compose the DIST product suite: 1) the **DIST-ALERT** product, capturing vegetation disturbance at the cadence of HLS sampling (2-3 days); and 2) the **DIST-ANN** product, summarizing the confirmed changes of the DIST-ALERT products from previous calendar year. 

This notebook provides a step-by-step workflow visualizing **DIST-ANN** raster layers for the 2022 calendar year. An analogous notebook for the **DIST-ALERT** product may be accessed [here](https://github.com/OPERA-Cal-Val/OPERA_Applications/blob/main/DIST/Wildfire/Intro_To_DIST.ipynb).

### Metadata
---

HLS products provide surface reflectance (SR) data from the Operational Land Imager (OLI) aboard the Landsat-8 remote sensing satellite and the Multi-Spectral Instrument (MSI) aboard the Sentinel-2 A/B remote sensing satellite. HLS products are distributed over projected map coordinates aligned with the Military Grid Reference System (MGRS). Each tile covers 109.8 kilometers squared divided into 3660 rows and 3660 columns at 30 meter pixel spacing. Each tile overlaps neighbors by 4900 meters in each direction.

### Raster Layers
___

The **DIST-ANN** product is distributed as a set of 16 Cloud-Optimized GeoTIFF (COG) files to enable download of only particular layers of interest to a given user. All L3 DIST layers are stored in files following GeoTIFF format specifications. Details specific to the available raster layers and their properties are available in the [OPERA DIST Product Specifications Document](https://d2pn8kiwq2w21t.cloudfront.net/documents/ProductSpec_DIST_HLS.pdf).

In [ ]:
# Specify filepath location of OPERA DIST_ANN tile and desired bands (Note: bandlist is not comprehensive of all available layers)
data_dir = 'https://glad.umd.edu/projects/opera/SEP/DIST-ANN/10/T/E/M/2022/OPERA_L3_DIST-ANN-HLS_T10TEM_2022_2023136T210508Z_30_v0_'
bandlist = ['VEG-DIST-STATUS', 'VEG-HIST', 'VEG-IND-MAX','VEG-ANOM-MAX', 'VEG-DIST-CONF', 'VEG-DIST-DATE', 
            'VEG-DIST-COUNT', 'VEG-DIST-DUR', 'VEG-LAST-DATE', 'GEN-DIST-STATUS', 'GEN-ANOM-MAX', 'GEN-DIST-CONF',
            'GEN-DIST-DATE', 'GEN-DIST-COUNT', 'GEN-DIST-DUR', 'GEN-LAST-DATE']
bandpath = f"{data_dir}%s.tif"

In [ ]:
# Create geocube of stacked bands
da, crs = stack_bands(bandpath, bandlist)

# Create basemap
base = gv.tile_sources.EsriTerrain.opts(width=1000, height=1000, padding=0.1)

## **Band 1: Vegetation Disturbance Status (VEG-DIST-STATUS)**
***

**Data Type:** UInt8<br>
**Description:** Status of confirmed disturbance, current provisional disturbance, and no disturbance.<br>

In [ ]:
base = gv.tile_sources.EsriNatGeo.opts(width=1000, height=1000, padding=0.1)
veg_dist_status = da.z.where(da['z']!=255).sel({'band':1})

color_key = {
    "Confirmed, <50% ongoing": "#fcbea5",
    "Confirmed, ≥50% ongoing": "#fb7050",
    "Confirmed, <50% completed": "#ea372a",
    "Confirmed, ≥50% completed": "#67000d",
}
levels = 4
ticks = [2.5, 3.5, 4.5, 5.5]
ticker = FixedTicker(ticks=ticks)
labels = dict(zip(ticks, color_key))

veg_dist_status.where(veg_dist_status!=0).hvplot.image(x='longitude', 
                             y='latitude', 
                             crs=crs, 
                             rasterize=True,
                             dynamic=True, 
                             aspect='equal', 
                             frame_width=500, 
                             frame_height=500,
                             clim=(2,6), alpha=0.8).opts(title=f"VEG_DIST_STATUS", xlabel='Longitude', ylabel='Latitude',
                                             color_levels = levels, cmap=tuple(color_key.values()),
                                             colorbar_opts={'ticker': ticker, 'major_label_overrides':labels}) * base

**Layer Values:**<br> 
* **0:** No disturbance<br>
* **2:** Confirmed disturbance with vegetation cover change <50% (ongoing) <br>
* **4:** Confirmed disturbance with vegetation cover change ≥50% (ongoing) <br>
* **5:** Confirmed disturbance with vegetation cover change <50% (completed) <br>
* **6:** Confirmed disturbance with vegetation cover change ≥50% (completed)  <br>
* **255:** NoData <br> 

## **Band 4: Maximum Vegetation Anomaly Value (VEG_ANOM_MAX)**
***

**Data Type:** UInt8<br>
**Description:** Difference between historical vegetation cover and vegetation cover at the date of maximum decrease (vegetation loss of 0- 100%). This layer can be used to threshold vegetation disturbance per a given sensitivity (e.g. disturbance of ≥20% vegetation cover loss).<br>

In [ ]:
base = gv.tile_sources.EsriNatGeo.opts(width=1000, height=1000, padding=0.1)
veg_anom_max = da.z.where(da['z']!=255).sel({'band':4})
veg_anom_max.where(veg_anom_max!=0).hvplot.image(x='longitude', 
                          y='latitude', 
                          crs=crs, 
                          rasterize=True, 
                          dynamic=True, 
                          aspect='equal', 
                          frame_width=500, 
                          frame_height=500, 
                          cmap='hot', 
                          clabel='Vegetation Loss (%)',
                          clim=(0,100), alpha=0.8).opts(title=f"VEG_ANOM_MAX", xlabel='Longitude', ylabel='Latitude').redim.nodata(value=255) * base

**Layer Values:**<br> 
* **0-100:** Maximum loss of percent vegetation<br>
* **255:** NoData <br>


## **Band 5: Vegetation Disturbance Confidence (VEG_DIST_CONF)**
***

**Data Type:** UInt16<br>
**Description:** Mean anomaly value since initial anomaly detection times the number of loss anomalies squared, until the anniversary date is reached, or a fixed number of consecutive non- anomalies are observed.<br>

In [ ]:
base = gv.tile_sources.EsriNatGeo.opts(width=1000, height=1000, padding=0.1)
veg_dist_confidence = da.z.where(da['z']!=255).sel({'band':5})
veg_dist_confidence.where(veg_dist_confidence!=0).hvplot.image(x='longitude', 
                             y='latitude', 
                             crs=crs, 
                             rasterize=True, 
                             dynamic=True, 
                             aspect='equal', 
                             frame_width=500, 
                             frame_height=500, 
                             cmap='blues',
                             clabel='Confidence Units', 
                             alpha=0.8).opts(title=f"VEG_DIST_CONFIDENCE", clim=(34,32000), colorbar_opts={'ticker': FixedTicker(ticks=[0, 10000, 20000, 30000])}, xlabel='Longitude', ylabel='Latitude') * base

**Layer Values:**<br> 
* **-1:** NoData <br>
* **0:** No disturbance <br>
* **>0:** Disturbance confidence <br>

## **Band 6: Date of Initial Vegetation Disturbance (VEG_DIST_DATE)**
***

**Data Type:** Int16<br>
**Description:** Day of first loss anomaly detection in the last year, denoted as the number of days since December 31st, 2020.<br>

In [ ]:
veg_dist_date = da.z.where(da['z']!=-1).sel({'band':6})
veg_dist_date.where(veg_dist_date!=0).hvplot.image(x='longitude', 
                           y='latitude', 
                           crs=crs, 
                           rasterize=True, 
                           dynamic=True, 
                           aspect='equal', 
                           frame_width=500, 
                           frame_height=500, 
                           cmap='inferno',
                           clabel='Days since 1/1/22', 
                           alpha=0.8).opts(title=f"VEG_DIST_DATE", xlabel='Longitude', ylabel='Latitude',clim=(0, 592)) * base

**Layer Values:**<br> 
* **-1:** NoData <br>
* **0:** No disturbance <br>
* **>0:** Day of first loss anomaly detection <br>

## **Band 7: Number of Vegetation Anomalies (VEG-DIST-COUNT)**
***

**Data Type:** UInt8<br>
**Description:** Total number of observations with anomalous low vegetation since initial anomaly detection (inclusive). Maximum of 254.<br>

In [ ]:
base = gv.tile_sources.EsriNatGeo.opts(width=1000, height=1000, padding=0.1)
veg_dist_count = da.z.where(da['z']!=255).sel({'band':7})
veg_dist_count.where(veg_dist_count!=0).hvplot.image(x='longitude', 
                             y='latitude', 
                             crs=crs, 
                             rasterize=True, 
                             dynamic=True, 
                             aspect='equal', 
                             frame_width=500, 
                             frame_height=500, 
                             cmap='cividis',
                             clabel='Number of Anomolaies Observed',
                             alpha=0.8).opts(title=f"VEG_DIST_COUNT", clim=(0,254), colorbar_opts={'ticker': FixedTicker(ticks=[0, 50, 100, 150, 200, 250])}, xlabel='Longitude', ylabel='Latitude') * base

**Layer Values:**<br> 
* **0:** No disturbance anomalies <br>
* **1-254:** Count of disturbance anomalies <br>
* **255:** NoData <br>

## **Band 8: Vegetation Disturbance Duration (VEG-DIST-DUR)**
***

**Data Type:** UInt16<br>
**Description:** Number of days of ongoing loss anomalies since initial anomaly detection (inclusive). Maximum duration is one year.<br>

In [ ]:
base = gv.tile_sources.EsriNatGeo.opts(width=1000, height=1000, padding=0.1)
veg_dist_dur = da.z.where(da['z']!=-1).sel({'band':8})
veg_dist_dur.where(veg_dist_dur!=0).hvplot.image(x='longitude', 
                             y='latitude', 
                             crs=crs, 
                             rasterize=True, 
                             dynamic=True, 
                             aspect='equal', 
                             frame_width=500, 
                             frame_height=500, 
                             cmap='magma_r',
                             clabel='Days',
                             alpha=0.8).opts(title=f"VEG_DIST_DUR", clim=(0,365), colorbar_opts={'ticker': FixedTicker(ticks=[0, 50, 100, 150, 200, 250, 300, 350])}, xlabel='Longitude', ylabel='Latitude') * base

**Layer Values:**<br> 
* **-1:** NoData <br>
* **0-366:** Number of days from first disturbance anomaly to the most recent disturbance anomaly detection <br>

## Conclusion
This notebook provides a basic workflows for loading and visualizing raster layers of the OPERA **DIST-ANN** product, a near-global, pixel-wise summary of vegetation loss for the 2022 calendar year. 